In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project='resonant-forge-338500')

Each Kaggle user can scan 5TB every 30 days for free. Once you hit that limit, you'll have to wait for it to reset.

### **Google's public datasets hosted on bigquery-public-data**

## SAMPLE Datasets:

## Github_nested 

* Table size 1.58 GB

* Number of rows 2,541,639

* Created 15 Mar 2016, 03:46:44 UTC+10:30

In [ ]:
# Construct a reference to the "sample" dataset
dataset_ref = client.dataset("samples", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "github_nested" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are two github sample datasets!)
for table in tables:  
    print(table.table_id)

In [ ]:
# Preview the first five lines of the table:                       (Print table schema)
def table_preview(client, project = 'bigquery-public-data',dataset_id = 'samples', table_id = 'github_nested'): 
    
    # Construct a reference to the dataset
    dataset_ref = client.dataset(dataset_id, project=project)
    # Construct a reference to the table
    table_ref = dataset_ref.table(table_id)
    # API request - fetch the table
    table = client.get_table(table_ref)

    # Iterate over table properties
    result = ["{0} {1}".format(schema.name,schema.field_type) for schema in table.schema]
    
    #print(result)
    
    return client.list_rows(table, max_results=5).to_dataframe()
    
github_nested   = table_preview(client)
github_timeline = table_preview(client, table_id = 'github_timeline')

In [ ]:
github_nested

# Github_timeline

In [ ]:
github_timeline

In [7]:
# Construct a reference to the dataset
dataset_ref = client.dataset('day', project='githubarchive')
# Construct a reference to the table
table_ref = dataset_ref.table('20210616')

# This query randomly extracts 10% of the table.
query = """
        SELECT * 
        FROM `githubarchive.day.20210616` 
        TABLESAMPLE SYSTEM (10 PERCENT)
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} GB.".format(dry_run_query_job.total_bytes_processed/1000000000))

This query will process 1.610086675 GB.


In [8]:

# Only run the query if it's less than limit
limit = 1000*1000*1000*10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=limit)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
df = safe_query_job.to_dataframe()

df.to_csv('/kaggle/working/20210616.csv',index=False)

In [12]:
df

,type,public,payload,repo,actor,org,created_at,id,other
0,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 232829668, 'name': 'YuDGang/FinderAPP',...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:07:15+00:00,16808180634,"{""actor"":{""display_login"":""dependabot""}}"
1,PullRequestEvent,True,"{""action"":""opened"",""number"":15,""pull_request"":...","{'id': 232892432, 'name': 'OPerel/tris', 'url'...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:08:12+00:00,16808193174,"{""actor"":{""display_login"":""dependabot""}}"
2,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 232919029, 'name': 'fecsaba/assignment8...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:08:33+00:00,16808197798,"{""actor"":{""display_login"":""dependabot""}}"
3,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 233182718, 'name': 'mifeng235/blog', 'u...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:14:13+00:00,16808271830,"{""actor"":{""display_login"":""dependabot""}}"
4,PullRequestEvent,True,"{""action"":""opened"",""number"":58,""pull_request"":...","{'id': 202710291, 'name': 'rishiosaur/hoot', '...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:17:08+00:00,16808309340,"{""actor"":{""display_login"":""dependabot""}}"
...,...,...,...,...,...,...,...,...,...
348348,DeleteEvent,True,"{""ref"":""2018.2.1"",""ref_type"":""tag"",""pusher_typ...","{'id': 2209077, 'name': 'JetBrains/MPS', 'url'...","{'id': 249955, 'login': 'SergeyZh', 'gravatar_...","{'id': 878437, 'login': 'JetBrains', 'gravatar...",2021-06-16 14:05:09+00:00,16804339027,"{""actor"":{""display_login"":""SergeyZh""}}"
348349,ForkEvent,True,"{""forkee"":{""id"":377517137,""node_id"":""MDEwOlJlc...","{'id': 201318681, 'name': 'LaunchCodeEducation...","{'id': 83477652, 'login': 'MFREERKSEN', 'grava...","{'id': 16672273, 'login': 'LaunchCodeEducation...",2021-06-16 14:05:27+00:00,16804344733,"{""actor"":{""display_login"":""MFREERKSEN""}}"
348350,PullRequestReviewEvent,True,"{""action"":""created"",""review"":{""id"":685216554,""...","{'id': 310305784, 'name': 'Tejas1510/Hacking-S...","{'id': 63305824, 'login': 'GowthamGoush', 'gra...",None,2021-06-16 14:05:47+00:00,16804350486,"{""actor"":{""display_login"":""GowthamGoush""}}"
348351,PushEvent,True,"{""push_id"":7339868238,""size"":1,""distinct_size""...","{'id': 377514072, 'name': 'iwata008/Covid', 'u...","{'id': 54686541, 'login': 'iwata008', 'gravata...",None,2021-06-16 14:05:55+00:00,16804353009,"{""actor"":{""display_login"":""iwata008""}}"


In [13]:
df['type'].unique()

array(['CreateEvent', 'PullRequestEvent', 'DeleteEvent',
       'IssueCommentEvent', 'PushEvent', 'WatchEvent', 'ForkEvent',
       'MemberEvent', 'CommitCommentEvent', 'IssuesEvent', 'PublicEvent',
       'PullRequestReviewEvent', 'ReleaseEvent', 'GollumEvent',
       'PullRequestReviewCommentEvent'], dtype=object)

In [23]:
print("The dataset contains {} distinct event types.".format(df['type'].unique().size))

The dataset contains 15 distinct event types.


In [124]:
CreateEvent = df[df['type'] == 'CreateEvent']

In [125]:
CreateEvent

,type,public,payload,repo,actor,org,created_at,id,other
0,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 232829668, 'name': 'YuDGang/FinderAPP',...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:07:15+00:00,16808180634,"{""actor"":{""display_login"":""dependabot""}}"
2,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 232919029, 'name': 'fecsaba/assignment8...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:08:33+00:00,16808197798,"{""actor"":{""display_login"":""dependabot""}}"
3,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 233182718, 'name': 'mifeng235/blog', 'u...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:14:13+00:00,16808271830,"{""actor"":{""display_login"":""dependabot""}}"
8,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 234036762, 'name': '15195986013/preact-...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:44:41+00:00,16808669925,"{""actor"":{""display_login"":""dependabot""}}"
10,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 234159481, 'name': 'euqueme/re-former',...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 18:46:24+00:00,16808692874,"{""actor"":{""display_login"":""dependabot""}}"
...,...,...,...,...,...,...,...,...,...
348326,CreateEvent,True,"{""ref"":""master"",""ref_type"":""branch"",""master_br...","{'id': 377515385, 'name': 'tfasano1/circumpunc...","{'id': 54379134, 'login': 'tfasano1', 'gravata...",None,2021-06-16 14:00:14+00:00,16804247061,"{""actor"":{""display_login"":""tfasano1""}}"
348333,CreateEvent,True,"{""ref"":""user_administration"",""ref_type"":""branc...","{'id': 377228341, 'name': 'CamClrt/p13', 'url'...","{'id': 57750666, 'login': 'CamClrt', 'gravatar...",None,2021-06-16 14:01:58+00:00,16804280653,"{""actor"":{""display_login"":""CamClrt""}}"
348337,CreateEvent,True,"{""ref"":""st-arturs-arutjunovs-lesson-8-level-2-...","{'id': 359137489, 'name': 'konstantins-buts/jg...","{'id': 82509041, 'login': 'arturwrls', 'gravat...",None,2021-06-16 14:02:20+00:00,16804287438,"{""actor"":{""display_login"":""arturwrls""}}"
348346,CreateEvent,True,"{""ref"":""dependabot/npm_and_yarn/postcss-7.0.36...","{'id': 220922034, 'name': 'WichoEguia/Proyecto...","{'id': 49699333, 'login': 'dependabot[bot]', '...",None,2021-06-16 14:05:04+00:00,16804337544,"{""actor"":{""display_login"":""dependabot""}}"


In [126]:
def data_overview (data_frame):
    # Overview of total values, missing values and datatypes.
    Values = data_frame.notnull().sum()
    null_counts = data_frame.isnull().sum().sort_values(ascending=False)
    null_percentage = (null_counts/len(data_frame)*100).round(1)
    data_types = data_frame.dtypes

    missing_values_table = pd.concat([Values, null_counts, null_percentage, data_types], axis=1)
    missing_values_table = missing_values_table.rename(columns={0: 'Total Values', 1: "Missing Values", 2: "Nulls (%)", 3: "Data Types"})

    return print(missing_values_table)

In [127]:
data_overview(CreateEvent)

            Total Values  Missing Values  Nulls (%)           Data Types
type               75565               0        0.0               object
public             75565               0        0.0                 bool
payload            75565               0        0.0               object
repo               75565               0        0.0               object
actor              75565               0        0.0               object
org                 7363           68202       90.3               object
created_at         75565               0        0.0  datetime64[ns, UTC]
id                 75565               0        0.0               object
other              75565               0        0.0               object


In [128]:
columns = ['payload','repo','actor','other']
for column in columns:
    print(CreateEvent.at[0,column])
    print(type(CreateEvent.at[0,column]))

{"ref":"dependabot/npm_and_yarn/postcss-7.0.36","ref_type":"branch","master_branch":"master","description":"Finder - 摄影器材租赁平台","pusher_type":"user"}
<class 'str'>
{'id': 232829668, 'name': 'YuDGang/FinderAPP', 'url': 'https://api.github.com/repos/YuDGang/FinderAPP'}
<class 'dict'>
{'id': 49699333, 'login': 'dependabot[bot]', 'gravatar_id': '', 'avatar_url': 'https://avatars.githubusercontent.com/u/49699333?', 'url': 'https://api.github.com/users/dependabot[bot]'}
<class 'dict'>
{"actor":{"display_login":"dependabot"}}
<class 'str'>


In [129]:
import json

columns = ['payload','other']

# Convert to proper json format
def string_to_dict(dict_string):
    
    try:
        return json.loads(dict_string)
    
    except:
        
        return dict_string


for column in columns:
    CreateEvent[column] = CreateEvent[column].apply(string_to_dict)
    aux = CreateEvent[column].apply(pd.Series).add_prefix(column + ' : ')
    CreateEvent = CreateEvent.drop([column], axis=1)
    CreateEvent = pd.concat([CreateEvent, aux], axis=1)
    
columns = ['repo','actor']

for column in columns:
    aux = CreateEvent[column].apply(pd.Series).add_prefix(column + ' : ')
    CreateEvent = pd.concat([CreateEvent, aux], axis=1)
    CreateEvent = CreateEvent.drop([column], axis=1)

CreateEvent

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,type,public,org,created_at,id,payload : ref,payload : ref_type,payload : master_branch,payload : description,payload : pusher_type,other : actor,repo : id,repo : name,repo : url,actor : id,actor : login,actor : gravatar_id,actor : avatar_url,actor : url
0,CreateEvent,True,None,2021-06-16 18:07:15+00:00,16808180634,dependabot/npm_and_yarn/postcss-7.0.36,branch,master,Finder - 摄影器材租赁平台,user,{'display_login': 'dependabot'},232829668,YuDGang/FinderAPP,https://api.github.com/repos/YuDGang/FinderAPP,49699333,dependabot[bot],,https://avatars.githubusercontent.com/u/49699333?,https://api.github.com/users/dependabot[bot]
2,CreateEvent,True,None,2021-06-16 18:08:33+00:00,16808197798,dependabot/npm_and_yarn/postcss-7.0.36,branch,master,None,user,{'display_login': 'dependabot'},232919029,fecsaba/assignment8,https://api.github.com/repos/fecsaba/assignment8,49699333,dependabot[bot],,https://avatars.githubusercontent.com/u/49699333?,https://api.github.com/users/dependabot[bot]
3,CreateEvent,True,None,2021-06-16 18:14:13+00:00,16808271830,dependabot/npm_and_yarn/postcss-7.0.36,branch,master,个人博客,user,{'display_login': 'dependabot'},233182718,mifeng235/blog,https://api.github.com/repos/mifeng235/blog,49699333,dependabot[bot],,https://avatars.githubusercontent.com/u/49699333?,https://api.github.com/users/dependabot[bot]
8,CreateEvent,True,None,2021-06-16 18:44:41+00:00,16808669925,dependabot/npm_and_yarn/postcss-7.0.36,branch,master,None,user,{'display_login': 'dependabot'},234036762,15195986013/preact-netlify,https://api.github.com/repos/15195986013/preac...,49699333,dependabot[bot],,https://avatars.githubusercontent.com/u/49699333?,https://api.github.com/users/dependabot[bot]
10,CreateEvent,True,None,2021-06-16 18:46:24+00:00,16808692874,dependabot/npm_and_yarn/postcss-7.0.36,branch,Development,None,user,{'display_login': 'dependabot'},234159481,euqueme/re-former,https://api.github.com/repos/euqueme/re-former,49699333,dependabot[bot],,https://avatars.githubusercontent.com/u/49699333?,https://api.github.com/users/dependabot[bot]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348326,CreateEvent,True,None,2021-06-16 14:00:14+00:00,16804247061,master,branch,master,A webring of creative individuals who put God ...,user,{'display_login': 'tfasano1'},377515385,tfasano1/circumpunct,https://api.github.com/repos/tfasano1/circumpunct,54379134,tfasano1,,https://avatars.githubusercontent.com/u/54379134?,https://api.github.com/users/tfasano1
348333,CreateEvent,True,None,2021-06-16 14:01:58+00:00,16804280653,user_administration,branch,develop,None,user,{'display_login': 'CamClrt'},377228341,CamClrt/p13,https://api.github.com/repos/CamClrt/p13,57750666,CamClrt,,https://avatars.githubusercontent.com/u/57750666?,https://api.github.com/users/CamClrt
348337,CreateEvent,True,None,2021-06-16 14:02:20+00:00,16804287438,st-arturs-arutjunovs-lesson-8-level-2-task-6,branch,main,JavaGuru - Java 1 Online Course Group - Tuesda...,user,{'display_login': 'arturwrls'},359137489,konstantins-buts/jg-java-1-online-spring-april...,https://api.github.com/repos/konstantins-buts/...,82509041,arturwrls,,https://avatars.githubusercontent.com/u/82509041?,https://api.github.com/users/arturwrls
348346,CreateEvent,True,None,2021-06-16 14:05:04+00:00,16804337544,dependabot/npm_and_yarn/postcss-7.0.36,branch,master,Blog básico,user,{'display_login': 'dependabot'},220922034,WichoEguia/ProyectoTSCI,https://api.github.com/repos/WichoEguia/Proyec...,49699333,dependabot[bot],,https://avatars.githubusercontent.com/u/49699333?,https://api.github.com/users/dependabot[bot]
